<a href="https://colab.research.google.com/github/SteveChengChen/data-analysis/blob/main/financial_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `ChengChen` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

In [2]:
!pip install datasets
!pip install regex
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00


In [3]:
import re
import json
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset

In [4]:
ds_banking = load_dataset("talkmap/banking-conversation-corpus")
ds_fintalk = load_dataset("ceadar-ie/FinTalk-19k")
ds_fiqa = load_dataset("bilalRahib/fiqa-personal-finance-dataset")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/366 [00:00<?, ?B/s]

banking_300k.csv:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5532112 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/19111 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train-00000-of-00001.parquet:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1987 [00:00<?, ? examples/s]

In [5]:
print(ds_banking["train"][0])  # Inspect first row
df_banking = ds_banking["train"].to_pandas()
print(df_banking.head(3))

{'conversation_id': '2b6544c382e6423b96785c1a135d8e95', 'speaker': 'agent', 'date_time': '2023-09-06T14:33:33+00:00', 'text': 'Good morning, thank you for calling Union Financial. My name is Monroe, how can I assist you today?'}
                    conversation_id speaker                         date_time  \
0  2b6544c382e6423b96785c1a135d8e95   agent         2023-09-06T14:33:33+00:00   
1  2b6544c382e6423b96785c1a135d8e95  client  2023-09-06T14:33:41.307692+00:00   
2  2b6544c382e6423b96785c1a135d8e95   agent  2023-09-06T14:33:50.538461+00:00   

                                                text  
0  Good morning, thank you for calling Union Fina...  
1  Hi Monroe, my name is Della. I need help manag...  
2  Of course, Della. I'd be happy to help you wit...  


In [6]:
print(ds_fintalk["train"][0])  # Inspect first row
df_fintalk = ds_fintalk["train"].to_pandas()
print(df_fintalk.head(3))

{'instruction': 'What are some job ideas for a 62-year-old man with limited skills and qualifications, low confidence, and a failed business?', 'response': 'Truck driving can be a lucrative career and can provide the opportunity to see different parts of the country.', 'context': 'The person asking the question is a relative of the 62-year-old man and is looking for job ideas to help him earn income and support himself before he is eligible for the age pension.', 'tag': 'Personal Finance'}
                                         instruction  \
0  What are some job ideas for a 62-year-old man ...   
1  What is an appropriate way to respond when a c...   
2  How can you learn to live with accumulated wea...   

                                            response  \
0  Truck driving can be a lucrative career and ca...   
1  Respecting coworkers' wishes is appropriate. A...   
2  The struggle with managing wealth and feeling ...   

                                             context   

In [7]:
print(ds_fiqa["train"][0])     # Inspect first row
df_fiqa = ds_fiqa["train"].to_pandas()
print(df_fiqa.head(3))

{'input': 'Does U.S. tax code call for small business owners to count business purchases as personal income?', 'output': 'I am going to keep things very simple and explain the common-sense reason why the accountant is right: Also, my sister in law owns a small restaurant, where they claim their accountant informed them of the same thing, where a portion of their business purchases had to be counted as taxable personal income. In this case, they said their actual income for the year (through their paychecks) was around 40-50K, but because of this detail, their taxable income came out to be around 180K, causing them to owe a huge amount of tax (30K ish). Consider them and a similarly situated couple that didn\'t make these purchases. Your sister in law is better off in that she has the benefit of these purchases (increasing the value of her business and her expected future income), but she\'s worse off because she got less pay. Presumably, she thought this was a fair trade, otherwise she

In [8]:
print("Banking corpus rows:", ds_banking["train"].num_rows)
print("FinTalk-19k rows:", ds_fintalk["train"].num_rows)
print("FiQA rows:", ds_fiqa["train"].num_rows)

Banking corpus rows: 5532112
FinTalk-19k rows: 19111
FiQA rows: 1987


In [9]:
def clean_text(text: str) -> str:
    """
    Perform basic text normalization steps:
    - Strip leading/trailing spaces
    - Replace multiple spaces/newlines with a single space
    """
    # Convert to string (in case it's not)
    text = str(text)
    # Strip whitespace at ends
    text = text.strip()
    # Replace multiple newlines/tabs with a space
    text = re.sub(r'[\r\n\t]+', ' ', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text

def anonymize_text(text: str) -> str:
    """
    Mask or remove potentially sensitive personal data such as:
      - Email addresses
      - Phone numbers
      - Long digit sequences (credit card, account #, etc.)
      - Simple name detection in patterns like "My name is ...", "I am ..."
      - Address keywords (e.g., 'street', 'road', etc.)
    """

    # 1) Email addresses
    text = re.sub(
        r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',
        '[EMAIL_MASKED]',
        text
    )

    # 2) Phone numbers (very rough)
    #    Matches common patterns like +1 555-123-4567, (555) 123-4567, etc.
    phone_pattern = r'(\+?\d{1,3}[-.\s]?\(?\d{1,4}\)?[-.\s]?\d{3}[-.\s]?\d{4,6})'
    text = re.sub(phone_pattern, '[PHONE_MASKED]', text)

    # 3) Long digit sequences (e.g. account numbers, credit cards, SSNs).
    #    Mask sequences of 6 or more consecutive digits.
    text = re.sub(r'\b\d{6,}\b', '[DIGITS_MASKED]', text)

    # 4) Simple name detection (very naive)
    #    Looks for patterns "my name is X" or "I am X"
    text = re.sub(
        r'\b(my name is|i am)\s+[A-Z][a-zA-Z]+',
        r'\1 [NAME_MASKED]',
        text,
        flags=re.IGNORECASE
    )

    # 5) Address keywords (extremely naive)
    address_keywords = r'(address|street|st\.|road|rd\.|apartment|apt\.|PO Box)'
    text = re.sub(address_keywords, '[ADDRESS_MASKED]', text, flags=re.IGNORECASE)

    return text

def create_banking_data():
    """
    Loads the 'talkmap/banking-conversation-corpus' dataset,
    groups rows by conversation_id, and returns a list of
    conversations in a unified role-based format (JSON-like).
    Each conversation starts with a system message.
    """
    ds_banking = load_dataset("talkmap/banking-conversation-corpus")
    df_banking = ds_banking["train"].to_pandas()

    dict_of_convos = {}

    for i, row in df_banking.iterrows():
        conv_id = row["conversation_id"]
        if conv_id not in dict_of_convos:
            dict_of_convos[conv_id] = []

        speaker = row["speaker"]
        if speaker == "agent":
            role = "assistant"
        elif speaker == "client":
            role = "user"
        else:
            role = "assistant"

        # Clean & anonymize
        original_text = str(row["text"])
        cleaned = clean_text(original_text)
        anonymized = anonymize_text(cleaned)

        dict_of_convos[conv_id].append({
            "role": role,
            "content": anonymized
        })

    unified_banking_data = []
    for conv_id, messages in dict_of_convos.items():
        # Insert a system message at the beginning
        system_msg = {
            "role": "system",
            "content": "You are a helpful banking assistant. Provide polite, concise answers."
        }
        messages.insert(0, system_msg)

        conversation_dict = {
            "id": conv_id,
            "messages": messages
        }
        unified_banking_data.append(conversation_dict)

    return unified_banking_data

def create_fintalk_data():
    """
    Loads the 'ceadar-ie/FinTalk-19k' dataset,
    converts each row (instruction + response + context) into
    a short multi-turn conversation in role-based format.
    """
    ds_fintalk = load_dataset("ceadar-ie/FinTalk-19k")
    df_fintalk = ds_fintalk["train"].to_pandas()

    unified_fintalk_data = []

    for i, row in df_fintalk.iterrows():
        conv_id = f"fintalk-{i}"
        instruction = clean_text(row["instruction"])
        response = clean_text(row["response"])
        context = clean_text(row.get("context", ""))

        # Anonymize
        instruction = anonymize_text(instruction)
        response = anonymize_text(response)
        context = anonymize_text(context)

        messages = []
        if context:
            # Put context in a system role
            messages.append({
                "role": "system",
                "content": f"Context: {context}"
            })

        messages.append({
            "role": "user",
            "content": instruction
        })
        messages.append({
            "role": "assistant",
            "content": response
        })

        unified_fintalk_data.append({
            "id": conv_id,
            "messages": messages
        })

    return unified_fintalk_data

def create_fiqa_data():
    """
    Loads 'bilalRahib/fiqa-personal-finance-dataset',
    converts each QA pair into a short 2-turn conversation
    (user => assistant).
    """
    ds_fiqa = load_dataset("bilalRahib/fiqa-personal-finance-dataset")
    df_fiqa = ds_fiqa["train"].to_pandas()

    unified_fiqa_data = []

    for i, row in df_fiqa.iterrows():
        conv_id = f"fiqa-{i}"
        question = clean_text(row["input"])
        answer = clean_text(row["output"])

        question = anonymize_text(question)
        answer = anonymize_text(answer)

        messages = [
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer}
        ]

        unified_fiqa_data.append({
            "id": conv_id,
            "messages": messages
        })

    return unified_fiqa_data

def main():
    # 1. Banking Data
    print("Loading & converting the Banking Conversation Corpus...")
    unified_banking_data = create_banking_data()
    print(f"  -> Banking data conversations: {len(unified_banking_data)}")

    # 2. FinTalk Data
    print("\nLoading & converting FinTalk-19k...")
    unified_fintalk_data = create_fintalk_data()
    print(f"  -> FinTalk data conversations: {len(unified_fintalk_data)}")

    # 3. FiQA Data
    print("\nLoading & converting FiQA Personal Finance Dataset...")
    unified_fiqa_data = create_fiqa_data()
    print(f"  -> FiQA data conversations: {len(unified_fiqa_data)}")

    # Combine all
    print("\nCombining all conversations into one list...")
    all_data = unified_banking_data + unified_fintalk_data + unified_fiqa_data
    print(f"  -> Total combined conversations: {len(all_data)}")

    # Write out to JSONL
    output_file = "unified_dataset_anonymized.jsonl"
    with open(output_file, "w", encoding="utf-8") as f:
        for convo in all_data:
            f.write(json.dumps(convo, ensure_ascii=False) + "\n")

    print(f"Anonymized dataset saved to '{output_file}'.")

In [10]:
if __name__ == "__main__":
    main()

Loading & converting the Banking Conversation Corpus...
  -> Banking data conversations: 301822

Loading & converting FinTalk-19k...
  -> FinTalk data conversations: 19111

Loading & converting FiQA Personal Finance Dataset...


Repo card metadata block was not found. Setting CardData to empty.


  -> FiQA data conversations: 1987

Combining all conversations into one list...
  -> Total combined conversations: 322920
Anonymized dataset saved to 'unified_dataset_anonymized.jsonl'.
